In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os
import gc
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, log_evaluation, record_evaluation
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
#from sklearn.impute import IterativeImputer
import pickle
import optuna
from optuna.visualization import plot_slice, plot_param_importances
import shap
import random

gc.enable()

pd.options.display.max_columns = None
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

#pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [2]:
path = 'I:/Kaggle/jane-street-real-time-market-data-forecasting/'

In [3]:
os.listdir(path)

['features.csv',
 'imputed_train_ffill.parquet',
 'kaggle_evaluation',
 'lags.parquet',
 'my_folder',
 'responders.csv',
 'sample_submission.csv',
 'team_folder',
 'test.parquet',
 'top_100000_rows_sorted_by_weight_descending.parquet',
 'top_10000_rows_sorted_by_weight_descending.parquet',
 'train.parquet']

In [12]:
train_df = pl.read_parquet(path + 'train.parquet/').drop(['responder_0', 'responder_1', 'responder_2', 'responder_3', 'responder_4', 'responder_5', 'responder_7', 'responder_8', 'partition_id']).select(pl.all().shrink_dtype())
print(train_df.shape)
train_df

(47127338, 84)


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,null,0.28118,0.269163,0.349028,-0.012596,-0.225932,null,-1.073602,null,null,-0.181716,null,null,null,0.564021,2.088506,0.832022,null,0.204797,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.775981
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,null,-1.512286,-1.414357,-1.823322,-0.082763,-0.184119,null,null,null,null,null,null,null,null,-10.835207,-0.002704,-0.621836,null,1.172836,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,0.703665
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,null,-0.320921,-0.95809,-2.436589,0.070999,-0.245239,null,null,null,null,null,null,null,null,-1.420632,-3.515137,-4.67776,null,0.535897,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,2.109352
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,null,0.544138,-0.087091,-1.500147,-0.201288,-0.038042,null,null,null,null,null,null,null,null,0.382074,2.669135,0.611711,null,2.413415,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,1.114137
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,null,-0.08842,-0.995003,-2.635336,-0.196461,-0.618719,null,n

In [5]:
feature_cols = [col for col in train_df.columns if 'feature' in col]
feature_cols

['feature_00',
 'feature_01',
 'feature_02',
 'feature_03',
 'feature_04',
 'feature_05',
 'feature_06',
 'feature_07',
 'feature_08',
 'feature_09',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_64',
 'feature_65',
 'feature_

In [6]:
train_scan = pl.scan_parquet(path + 'train.parquet/')
test_scan = pl.scan_parquet(path + 'test.parquet/')

In [7]:
train_symbol_ids_list = sorted(train_scan.select('symbol_id').unique().collect()['symbol_id'].to_list())
test_symbol_ids_list = sorted(test_scan.select('symbol_id').unique().collect()['symbol_id'].to_list())
unique_symbol_ids_list = sorted(list(set(train_symbol_ids_list + test_symbol_ids_list)))
unique_symbol_ids_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38]

In [10]:
def bfill_impute(df):
    df = df.clone()
    
    results = []
    for symbol_id in tqdm(unique_symbol_ids_list):
        symbol_id_df = df.filter(pl.col("symbol_id") == symbol_id)
        #display(symbol_id_df)
        symbol_id_df = symbol_id_df.fill_null(strategy="forward")
        #display(symbol_id_df)
        symbol_id_df = symbol_id_df.fill_null(strategy="backward")
        results.append(symbol_id_df)

        #return

    # Combine all backfilled DataFrames
    result_df = pl.concat(results).sort(by=['date_id', 'time_id', 'symbol_id'])
    return result_df

In [13]:
train_df = bfill_impute(train_df)

  0%|                                                                                           | 0/39 [00:00<?, ?it/s]

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1,0,0,1.749479,null,null,null,null,null,0.053447,2.192887,1.160708,-0.00983,11,7,76,-0.819115,1.403962,-0.098782,null,0.340309,null,-1.577595,-1.237946,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,1.48572,0.760141,2.145851,-0.021399,-0.016591,null,null,null,null,null,null,null,null,3.88576,7.287179,4.877959,null,2.20351,null,null,-0.850504,null,-1.543995,1.741718,null,2.035935,0.892778,-1.081097,null,null,null,null,null,-0.734145,0.276101,-0.576441,-0.799783,4.812901,0.393107,null,null,-0.226359,-0.327119,-0.31504,-0.385573,2.337418
1,1,0,1.749479,null,null,null,null,null,-0.029047,0.847413,0.769509,0.009426,11,7,76,-0.67729,2.703819,0.307246,null,0.361542,null,-1.508338,-1.414294,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,1.272551,0.986756,2.769935,0.028861,0.036394,null,null,null,null,null,null,null,null,3.062142,1.634271,3.670358,null,1.121471,null,null,-1.338483,null,-1.557803,1.283185,null,-0.10197,0.154968,-1.081097,null,null,null,null,null,-0.828776,3.034146,0.070293,-0.49952,3.384181,0.708746,null,null,-0.286174,-0.187674,-0.354506,-0.302946,2.492198
1,2,0,1.749479,null,null,null,null,null,0.017732,0.727187,1.153705,-0.0056,11,7,76,-0.587609,3.081821,0.2564,null,0.19876,null,-1.091023,-1.343286,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,0.09876,1.217269,2.32622,0.054985,0.054874,null,null,null,null,null,null,null,null,4.411406,1.200419,3.279326,null,1.229474,null,null,-2.004817,null,-1.917212,0.923784,null,-0.364484,0.043953,-1.081097,null,null,null,null,null,-0.528249,1.561701,0.044169,-0.697554,2.512118,0.71579,null,null,-0.32492,-0.306634,-0.342315,-0.33082,1.993902
1,3,0,1.749479,null,null,null,null,null,0.247528,1.072752,1.534643,-0.041765,11,7,76,-0.658146,1.82926,0.238465,null,0.01081,null,-1.370826,-1.561505,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,0.427911,1.390987,2.505615,0.059815,0.037799,null,null,null,null,null,null,null,null,2.351531,1.33464,2.359366,null,0.967031,null,null,0.252059,null,-1.972575,0.864815,null,-0.252517,0.055876,-1.081097,null,null,null,null,null,-0.720327,1.186939,0.029493,-0.728824,2.11122,0.556564,null,null,-0.242646,-0.302316,-0.253091,-0.313789,1.864082
1,4,0,1.749479,null,null,null,null,null,-0.024495,-0.506043,0.517926,0.006077,11,7,76,-0.984723,0.849526,0.356314,null,-0.123173,-0.03724,-1.412314,-1.341913,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,0.252309,1.170764,2.555653,0.0561,0.056733,null,null,null,null,null,null,null,null,2.886509,0.777438,3.265

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1,0,0,1.749479,null,null,null,null,null,0.053447,2.192887,1.160708,-0.00983,11,7,76,-0.819115,1.403962,-0.098782,null,0.340309,null,-1.577595,-1.237946,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,1.48572,0.760141,2.145851,-0.021399,-0.016591,null,null,null,null,null,null,null,null,3.88576,7.287179,4.877959,null,2.20351,null,null,-0.850504,null,-1.543995,1.741718,null,2.035935,0.892778,-1.081097,null,null,null,null,null,-0.734145,0.276101,-0.576441,-0.799783,4.812901,0.393107,null,null,-0.226359,-0.327119,-0.31504,-0.385573,2.337418
1,1,0,1.749479,null,null,null,null,null,-0.029047,0.847413,0.769509,0.009426,11,7,76,-0.67729,2.703819,0.307246,null,0.361542,null,-1.508338,-1.414294,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,1.272551,0.986756,2.769935,0.028861,0.036394,null,null,null,null,null,null,null,null,3.062142,1.634271,3.670358,null,1.121471,null,null,-1.338483,null,-1.557803,1.283185,null,-0.10197,0.154968,-1.081097,null,null,null,null,null,-0.828776,3.034146,0.070293,-0.49952,3.384181,0.708746,null,null,-0.286174,-0.187674,-0.354506,-0.302946,2.492198
1,2,0,1.749479,null,null,null,null,null,0.017732,0.727187,1.153705,-0.0056,11,7,76,-0.587609,3.081821,0.2564,null,0.19876,null,-1.091023,-1.343286,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,0.09876,1.217269,2.32622,0.054985,0.054874,null,null,null,null,null,null,null,null,4.411406,1.200419,3.279326,null,1.229474,null,null,-2.004817,null,-1.917212,0.923784,null,-0.364484,0.043953,-1.081097,null,null,null,null,null,-0.528249,1.561701,0.044169,-0.697554,2.512118,0.71579,null,null,-0.32492,-0.306634,-0.342315,-0.33082,1.993902
1,3,0,1.749479,null,null,null,null,null,0.247528,1.072752,1.534643,-0.041765,11,7,76,-0.658146,1.82926,0.238465,null,0.01081,null,-1.370826,-1.561505,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,0.427911,1.390987,2.505615,0.059815,0.037799,null,null,null,null,null,null,null,null,2.351531,1.33464,2.359366,null,0.967031,null,null,0.252059,null,-1.972575,0.864815,null,-0.252517,0.055876,-1.081097,null,null,null,null,null,-0.720327,1.186939,0.029493,-0.728824,2.11122,0.556564,null,null,-0.242646,-0.302316,-0.253091,-0.313789,1.864082
1,4,0,1.749479,null,null,null,null,null,-0.024495,-0.506043,0.517926,0.006077,11,7,76,-0.984723,0.849526,0.356314,null,-0.123173,-0.03724,-1.412314,-1.341913,1.039095,null,0.756488,0.432157,-1.626238,-0.137137,null,null,0.952274,-0.186008,0.127051,null,null,null,0.252309,1.170764,2.555653,0.0561,0.056733,null,null,null,null,null,null,null,null,2.886509,0.777438,3.265

  0%|                                                                                           | 0/39 [00:00<?, ?it/s]


In [ ]:
train_df

In [ ]:
train_df.null_count().sum_horizontal()

In [ ]:
def get_medians(df):
    medians_dict = {}
    for symbol_id in tqdm(unique_symbol_ids_list):
        symbol_id_df = df.filter(pl.col('symbol_id') == symbol_id)
        #for col in feature_cols:
        medians_dict[f'{symbol_id}'] = {col:symbol_id_df[col].median() for col in feature_cols}

    return medians_dict

In [ ]:
train_medians = get_medians(train_df)

In [ ]:
train_medians

In [ ]:
train_df.estimated_size() / 1e9

In [ ]:
models_path = path + 'my_folder/models/20250110_02/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [ ]:
train_df.write_parquet(models_path + 'imputed_train_ffill.parquet')

In [ ]:
previous_models_path = path + 'my_folder/models/20250109_03/'

In [ ]:
lgb_params_df = pd.read_csv(previous_models_path + 'lgb_params.csv')
lgb_params_df

In [ ]:
def lgb_sliding_window(train_data):#, optuna_n_trials):

    unique_date_ids = sorted(train_data['date_id'].unique())
    #unique_date_ids = [i for i in range(1110, train_data['date_id'].max()+1)]
    #date_ids_df = train_data['date_id'].to_frame()
    sample_df = train_data.sample(fraction=lgb_params_df['fraction'][0])
    unique_date_ids = sorted(sample_df['date_id'].unique())
    print(len(unique_date_ids))

    for date_id in unique_date_ids:
        val_window_size = lgb_params_df['val_window_size'][0]
        training_window_size = lgb_params_df['training_window_size'][0]
        #fraction = trial.suggest_float('fraction', 0.05, 0.3)
        #sample_df = train_data.sample(fraction=fraction)
        #unique_date_ids = [i for i in range(1110, sample_df['date_id'].max()+1)]
        #date_id = random.choice(unique_date_ids)

        test_date_id_df = sample_df.filter(pl.col('date_id') == date_id)

        val_date_id_cut_lower = date_id - val_window_size
        if val_date_id_cut_lower < 0:
            val_window_df = sample_df.filter(pl.col('date_id') <= val_window_size)
            val_window_df = val_window_df.join(test_date_id_df, on=['date_id', 'time_id', 'symbol_id'], how='anti')
        else:
            val_window_df = sample_df.filter((pl.col('date_id') >= val_date_id_cut_lower)&(pl.col('date_id') < date_id))
        
        training_date_id_cut_lower = val_date_id_cut_lower - training_window_size
        if training_date_id_cut_lower < 0:
            training_window_df = sample_df.filter(pl.col('date_id') <= val_window_size + training_window_size)
            training_window_df = training_window_df.join(test_date_id_df, on=['date_id', 'time_id', 'symbol_id'], how='anti')
            training_window_df = training_window_df.join(val_window_df, on=['date_id', 'time_id', 'symbol_id'], how='anti')
        else:
            training_window_df = sample_df.filter((pl.col('date_id') >= training_date_id_cut_lower)&(pl.col('date_id') < val_date_id_cut_lower))

        
        '''
        test_date_id_df = sample_df.filter(pl.col('date_id') == date_id)

        val_date_id_cut_lower = date_id - val_window_size
        val_window_df = sample_df.filter((pl.col('date_id') >= val_date_id_cut_lower)&(pl.col('date_id') < date_id))#.sample(fraction=fraction)

        training_date_id_cut_lower = val_date_id_cut_lower - training_window_size
        training_window_df = sample_df.filter((pl.col('date_id') >= training_date_id_cut_lower)&(pl.col('date_id') < val_date_id_cut_lower))#.sample(fraction=fraction).sort(by=['date_id', 'time_id', 'symbol_id'])
        '''
        
        '''
        print(date_id)
        print('this is training_window_df')
        display(training_window_df)
        print('training_window_size:', training_window_size)
        print('n unique in training_window_df:', training_window_df['date_id'].n_unique())

        print('this is val_window_df')
        display(val_window_df)
        print('val_window_size:', val_window_size)
        print('n unique in training_window_df:', val_window_df['date_id'].n_unique())

        print('this is test_date_id_df')
        display(test_date_id_df)
        print('n unique in test_date_id_df:', test_date_id_df['date_id'].n_unique())
        '''

        #training_window_df = training_window_df.sample(fraction=fraction)
                

        '''
        print('this is window_df')
        display(window_df)
        print('this is date_id_df')
        display(date_id_df)
        '''
    
        base_params = {
            'verbosity': -1,
            'device': 'gpu',
            'early_stopping_round': 20,
        }
    
        params_to_tune = {
            'learning_rate': lgb_params_df['learning_rate'][0],
            'max_depth': lgb_params_df['max_depth'][0],
            'min_data_in_leaf': lgb_params_df['min_data_in_leaf'][0],
            'num_leaves': lgb_params_df['num_leaves'][0],
            'min_gain_to_split': lgb_params_df['min_gain_to_split'][0],
            'lambda_l1': lgb_params_df['lambda_l1'][0],
            'lambda_l2': lgb_params_df['lambda_l2'][0],
            'feature_fraction': lgb_params_df['feature_fraction'][0],
        }
    
        model = LGBMRegressor(
            **base_params,
            **params_to_tune,
            n_estimators=100000
        )
    
        X_train = training_window_df.drop(['date_id', 'time_id', 'symbol_id', 'weight', 'responder_6']).select(pl.all().shrink_dtype()).to_pandas()
        X_val = val_window_df.drop(['date_id', 'time_id', 'symbol_id', 'weight', 'responder_6']).select(pl.all().shrink_dtype()).to_pandas()
        X_test = test_date_id_df.drop(['date_id', 'time_id', 'symbol_id', 'weight', 'responder_6']).select(pl.all().shrink_dtype()).to_pandas()
    
        y_train = training_window_df['responder_6'].to_pandas()
        y_val = val_window_df['responder_6'].to_pandas()
        y_test = test_date_id_df['responder_6'].to_pandas()
    
        weights_train = training_window_df['weight'].to_pandas()
        weights_val = val_window_df['weight'].to_pandas()
        weights_test = test_date_id_df['weight'].to_pandas()
    
        model.fit(X_train, y_train, sample_weight=weights_train, eval_set=[(X_train, y_train), (X_val, y_val)], eval_sample_weight=[weights_train, weights_val])#, callbacks=[log_evaluation(period=10)])
        
        test_preds = model.predict(X_test)

        test_score = r2_score(y_test, test_preds, sample_weight=weights_test)

        print('date_id is:', date_id)
        print('Test Weighted R2 score is:', test_score)

In [ ]:
lgb_sliding_window(train_df)#, 300)

In [ ]:
for param in lgb_study.best_params.keys():
    fig = plot_slice(lgb_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(lgb_study)

In [ ]:
lgb_study.best_params

In [ ]:
lgb_study.best_value

In [ ]:
for k, v in lgb_study.best_params.items():
    print(k, v)

In [ ]:
lgb_params_df = pd.DataFrame({k:[v] for k, v in lgb_study.best_params.items()})

In [ ]:
lgb_params_df

In [ ]:
lgb_params_df.to_csv(models_path + 'lgb_params.csv', index=False)